In [1]:
import kaggle_benchmarks as kbench
import pandas as pd

# 1. Prepare a sample dataset mimicking the structure of the MedQA-Ro dataset.
data_df = pd.read_csv("/kaggle/input/datasets/gpreda/medqaro-benchmark-for-medical-q-and-a-in-romanian/romedqa_val_dataset.csv")

# 1.1. Prepare a reduced sample
# 3nd batch
data_df = data_df.iloc[450:700]

# 1.2. Rename columns
data_df.columns = ["Nr", "Epicrisis", "Question", "Answer"]

# 1.3. Filter only payload
data_df = data_df[["Epicrisis", "Question", "Answer"]]


# 3. Define the main task to evaluate the entire dataset.
@kbench.task(name="evaluate_med_qa_dataset_batch_04")
def evaluate_med_qa_dataset_batch_04(llm, df: pd.DataFrame) -> tuple[int, int]:
    """Runs the medical QA evaluation on the entire dataset and returns the score."""
    with kbench.client.enable_cache():
        # Use .evaluate() to run the single_item_task on each row of the DataFrame.
        runs = single_romanian_med_qa.evaluate(
            llm=[llm],
            evaluation_data=df,
            n_jobs=8,
        )

    eval_df = runs.as_dataframe()

    # Handle cases where evaluation might fail or produce no results.
    if eval_df.empty or 'result' not in eval_df.columns:
        return 0, len(df)

    # Calculate the total number of correct answers.
    # The 'is_correct' field is extracted from the result dictionary of each run.
    correct_count = int(eval_df.result.str.get("is_correct").sum())
    total_count = len(df)

    return correct_count / total_count , total_count

# 4. Run the main evaluation task with the prepared DataFrame.
evaluate_med_qa_dataset_batch_04.run(kbench.llm, df=data_df)

BokehModel(combine_events=True, render_bundle={'docs_json': {'ef4329d5-3c64-41fc-b4ba-effd3b2312b6': {'version…